<a href="https://colab.research.google.com/github/samanthajmichael/ml_project/blob/main/notebooks/Base_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%capture
!pip install tensorflow ffmpeg-python opencv-python matplotlib

In [14]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [15]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [16]:
def load_and_preprocess_frame(frame_data, target_size=(224, 224)):
    nparr = np.frombuffer(frame_data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img.astype(np.float32) / 255.0
    return img

def data_generator(df, batch_size):
    while True:
        batch = df.sample(n=batch_size)
        frames = [load_and_preprocess_frame(frame_data) for frame_data in batch['frame_data']]
        yield np.array(frames)

def augment_frame(frame):
    frame = tf.image.random_flip_left_right(frame)
    frame = tf.image.random_brightness(frame, max_delta=0.1)
    frame = tf.image.random_contrast(frame, lower=0.9, upper=1.1)
    frame = tf.clip_by_value(frame, 0.0, 1.0)
    return frame

def create_base_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    projection_head = Dense(128, activation='relu')(x)
    return Model(inputs=base_model.input, outputs=projection_head)

def nt_xent_loss(z_i, z_j, temperature=0.5):
    z_i = tf.math.l2_normalize(z_i, axis=1)
    z_j = tf.math.l2_normalize(z_j, axis=1)
    similarity_matrix = tf.matmul(z_i, z_j, transpose_b=True) / temperature
    batch_size = tf.shape(z_i)[0]
    contrastive_labels = tf.range(batch_size)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=contrastive_labels, logits=similarity_matrix)
    return tf.reduce_mean(loss)

@tf.function
def train_step(model, optimizer, x):
    x_i = augment_frame(x)
    x_j = augment_frame(x)

    with tf.GradientTape() as tape:
        z_i = model(x_i)
        z_j = model(x_j)
        loss = nt_xent_loss(z_i, z_j)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train_contrastive_model(df, epochs=1, batch_size=15, steps_per_epoch=50):
    model = create_base_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    generator = data_generator(df, batch_size)

    for epoch in range(epochs):
        total_loss = 0

        for _ in range(steps_per_epoch):
            x = next(generator)
            loss = train_step(model, optimizer, x)
            total_loss += loss

        avg_loss = total_loss / steps_per_epoch
        print(f"Epoch {epoch+1}/{epochs}, Avg Loss: {avg_loss:.4f}")

        # Optional: Save model checkpoint
        if (epoch) % 10 == 0:
            model.save(f'/content/model_checkpoint_epoch_{epoch+1}.h5')

    return model

In [17]:
# Usage
df = pd.read_pickle("/content/drive/MyDrive/ML Project/middle_15min_frames.pkl")

In [18]:
print(df.info())
print(f"DataFrame shape: {df.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20700 entries, 0 to 20699
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   frame_number  20700 non-null  int64 
 1   frame_data    20700 non-null  object
dtypes: int64(1), object(1)
memory usage: 323.6+ KB
None
DataFrame shape: (20700, 2)


In [19]:
# Investigate frame numbers
print("\nFrame number statistics:")
print(df['frame_number'].describe())

# Check for duplicate frame numbers
duplicates = df['frame_number'].duplicated().sum()
print(f"\nNumber of duplicate frame numbers: {duplicates}")

# Display a few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(df.head())


Frame number statistics:
count     20700.000000
mean      92814.500000
std        5975.719622
min       82465.000000
25%       87639.750000
50%       92814.500000
75%       97989.250000
max      103164.000000
Name: frame_number, dtype: float64

Number of duplicate frame numbers: 0

First few rows of the DataFrame:
   frame_number                                         frame_data
0         82465  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1         82466  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2         82467  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3         82468  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4         82469  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


In [20]:
trained_model = train_contrastive_model(df, epochs=1, batch_size=15, steps_per_epoch=50)
trained_model.save('/content/trained_base_model.h5')
print("Training completed and model saved.")

Epoch 1/1, Avg Loss: 2.7077


Training completed and model saved.


In [21]:
trained_model

<Functional name=functional_3, built=True>